In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# vocab lay tu https://github.com/undertheseanlp/dictionary
# https://github.com/duyet/vietnamese-wordlist

In [2]:
import torch.nn as nn
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

In [11]:
dataset_path='vispamdetection_dataset'

data = pd.read_csv(dataset_path+'/reviews.csv/reviews.csv')
# vocab = pd.read_json('vocab/vocab.json', orient='index')
vocab = pd.read_json('test_vocab.json', orient='index')



In [4]:
data.head()

,link,rating,comment,label,spam_label
0,https://shopee.vn/Ví-nam-đựng-Card-Feasty-da-n...,5,Tuyệt vời cho các e bé nghe đáng kể vs anh đi ...,1,3
1,https://shopee.vn/Ví-nam-đựng-Card-Feasty-da-n...,5,Chiến dịch tiêm vaccine Covid-19 toàn dân bắt ...,1,3
2,https://shopee.vn/Ví-nam-đựng-Card-Feasty-da-n...,5,Nsnwnwnxnwkkxmmxmsmwmsmsxnndnwnxnsnwnznn ncn1m...,1,3
3,https://shopee.vn/Ví-nam-đựng-Card-Feasty-da-n...,2,"Chất lượng sản phẩm ko như quảng cáo, kich thu...",0,0
4,https://shopee.vn/Ví-nam-đựng-Card-Feasty-da-n...,5,"Giao hàng nhanh. Shop đóng gói cẩn thận, thế n...",0,0


In [12]:
vocab.head()

,0
A,5
ABC,6
ADN,7
AIDS,8
AK,9


In [22]:
test_dataset = data
vocab_dict = vocab.to_dict()[0]

In [15]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

tokens = tokenizer.tokenize(test_dataset['comment'][0])
print("Tokens:", tokens)

Tokens: ['Tuyệt', 'vời', 'cho', 'các', 'e', 'bé', 'nghe', 'đáng', 'kể', 'vs', 'anh', 'đi', 'ăn', 'tối', 'và', 'nghỉ', 'đêm', 'trên', 'máy', 'bay', 'à', 'em', 'muốn', 'nhờ', 'các', 'bạn', 'trẻ', 'mà', 'không', 'cần', 'thiết', 'để', 'tiến', 'tới', 'hôn', 'gái', 'đúng', 'là', 'không', 'thể', 'thiếu', 'của', 'bao', 'giờ', 'có', 'truyện', 'tranh', 'hài', 'hoà', 'hợp', 'ước', 'hẹn']


In [29]:
def encode_sentence(s, vocab):
    return [vocab.get(i, 0) for i in s.lower().split()]

encoded = []
i = 0
for comment in test_dataset['comment']:
    encoded.append(encode_sentence(comment, vocab_dict))
    i +=1
    if i ==128:
        break
print(len(encoded))

128


In [30]:
import torch
from torch.nn.utils.rnn import pad_sequence

# Example of indexed sentences

# Convert indexed sentences to tensors
sentence_tensors = [torch.tensor(sentence, dtype=torch.long) for sentence in encoded]

# Pad the sentences to the same length (use padding value 0)
padded_sentences = pad_sequence(sentence_tensors, batch_first=True, padding_value=0)

# Print the padded sentences
print("Padded Sentences:")
print(padded_sentences)
print(padded_sentences.size())


Padded Sentences:
tensor([[ 8719,  9350,  2292,  ...,     0,     0,     0],
        [ 2277,  3264,  8348,  ...,     0,     0,     0],
        [    0,     0,     0,  ...,     0,     0,     0],
        ...,
        [10117,  5365,  4215,  ...,     0,     0,     0],
        [    0,     0,     0,  ...,     0,     0,     0],
        [    0,     0,  6221,  ...,     0,     0,     0]])
torch.Size([128, 81])


In [32]:
import torch
import torch.nn as nn

vocab_size = 19870
embedding_dim = 128 

embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)


# Forward pass: Pass the padded tokens through the embedding layer
embedded_tokens = embedding_layer(padded_sentences)

# Print the output embeddings
# print("Embedded Tokens:")
# print(embedded_tokens)
print("Shape of Embedded Tokens:", embedded_tokens.shape)

Shape of Embedded Tokens: torch.Size([128, 81, 128])


In [35]:
import torch.nn as nn

lstm = nn.LSTM(input_size=embedding_dim, hidden_size=128, num_layers=1, batch_first=True)

lstm_output, (hidden_state, cell_state) = lstm(embedded_tokens)

print("LSTM Output Shape:", lstm_output.shape)  # (batch_size, seq_length, hidden_size)


LSTM Output Shape: torch.Size([128, 81, 128])


In [11]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.embedding(x)
        _, hidden = self.rnn(x)
        output = self.fc(hidden.squeeze(0))
        return torch.sigmoid(output)

In [12]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.lstm(x)
        output = self.fc(hidden.squeeze(0))
        return torch.sigmoid(output)

In [13]:
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.embedding(x)
        _, hidden = self.gru(x)
        output = self.fc(hidden.squeeze(0))
        return torch.sigmoid(output)

In [14]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10, device="cpu"):

    model = model.to(device)
    for epoch in range(epochs):
        
        model.train()
        train_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), labels.float())
                val_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss / len(train_loader):.4f}, Validation Loss: {val_loss / len(val_loader):.4f}")

In [15]:
vocab_size = len(vocab_dict) + 1  # Vocabulary size (+1 for unknown tokens)
embed_dim = 100  # Size of word embeddings
hidden_dim = 128  # Number of hidden units
output_dim = 1  # For binary classification

# Instantiate each model
rnn_model = RNNModel(vocab_size, embed_dim, hidden_dim, output_dim)
lstm_model = LSTMModel(vocab_size, embed_dim, hidden_dim, output_dim)
gru_model = GRUModel(vocab_size, embed_dim, hidden_dim, output_dim)


In [16]:
# Binary Cross Entropy Loss for binary classification
criterion = nn.BCELoss()

# Optimizers for each model
rnn_optimizer = optim.Adam(rnn_model.parameters(), lr=0.001)
lstm_optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)
gru_optimizer = optim.Adam(gru_model.parameters(), lr=0.001)
